In [1]:
import pandas as pd

lex_happiness = pd.read_csv("clean_happiness.csv", delimiter=",", dtype={"text": str, "affect": str, "stems": str})
lex_sadness = pd.read_csv("clean_sadness.csv", delimiter=",", dtype={"text": str, "affect": str, "stems": str})
lex_anger = pd.read_csv("clean_anger.csv", delimiter=",", dtype={"text": str, "affect": str, "stems": str})
lex_fear = pd.read_csv("clean_fear.csv", delimiter=",", dtype={"text": str, "affect": str, "stems": str})
list_happiness = lex_happiness["stems"].tolist()
list_sadness = lex_sadness["stems"].tolist()
list_anger = pd.Series(lex_anger["stems"].tolist())
list_fear = lex_fear["stems"].tolist()

emotions = ["happiness", "sadness", "anger", "fear"]
datasets = [lex_happiness, lex_sadness, lex_anger, lex_fear]

In [3]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")

In [3]:
for index, dataset in enumerate(datasets):
    new_dataset = dataset.copy()
    new_dataset = new_dataset.drop(labels=["stems", "affect"], axis=1)
    
    
    text_rows = [row[1]["text"] for row in dataset.iterrows()]
    doc = nlp(" ".join(text_rows))
    pos_rows = [token.pos for token in doc]
    new_dataset["pos"] = pos_rows
    
    
    for index2,row in new_dataset.iterrows():
        if row[2] != 84:
            new_dataset = new_dataset.drop(index=index2, axis=0)
    new_dataset = new_dataset.drop(labels="pos", axis=1)
    
    
    new_dataset.to_csv("clean_" + emotions[index] + "_adj.csv", index=False, float_format='%.3f')
    

In [6]:
import random
#doc = nlp("I ate a banana")
#displacy.render(doc, style="dep")
doc = nlp("This car is a car with lights")
highest_score = 0.29
displacy.render(nlp("The house is green"), style="dep")

bot_output_dict = {
    "text": [token.text for token in doc],
    "pos": [token.pos for token in doc],
    "dep": [token.dep_ for token in doc]
}

print([(token.text, token.pos, token.dep_) for token in doc])


if bot_output_dict["pos"][0] == 92:
    num_nouns = bot_output_dict["pos"].count(92) - 1
else: 
    num_nouns = bot_output_dict["pos"].count(92)
    
num_nouns *= highest_score
num_nouns = round(num_nouns)
    

indices = []

if num_nouns > 0:
    indices = [i for i, item in enumerate(bot_output_dict["text"]) 
               if bot_output_dict["pos"][i] == 92
              and (bot_output_dict["dep"][i] == "nsubj" 
                   or bot_output_dict["dep"][i] == "dobj"
                  or bot_output_dict["dep"][i] == "pobj")]

    print("i1:", indices)
    
    
    
# randomly select num_nouns from list
random.shuffle(indices)
indices_to_replace = indices[:num_nouns]

[('This', 90, 'det'), ('car', 92, 'nsubj'), ('is', 100, 'ROOT'), ('a', 90, 'det'), ('car', 92, 'attr'), ('with', 85, 'prep'), ('lights', 92, 'pobj')]
i1: [1, 6]


In [2]:
import pandas as pd
import spacy
# now look for an an adjective
lexicon = pd.read_csv("clean_happiness_adj.csv", delimiter=",", dtype={"text": str, "intensity": float}, float_precision='round_trip')
lexicon_text_list = lexicon["text"].tolist()
lexicon_intensity_list = lexicon["intensity"].tolist()


In [83]:
import collections
def insert_adjectives(bot_output, lex, list_text, list_score, highest_score):
    doc = nlp(bot_output)
    pos = [token.pos_ for token in doc]
    dep = [token.dep_ for token in doc]
    noun_indices = []
    
    for index, token in enumerate(doc):
        pass
        #print(index, token, pos[index], dep[index])
    
    # save indices of words that could receive an adjective
    noun_indices = [index for index, token in enumerate(doc) if pos[index] == "NOUN" and (dep[index] == "nsubj" or dep[index] == "dobj" or dep[index] == "pobj" or dep[index] == "attr")]
    if 0 in noun_indices: noun_indices.remove(0)
    noun_indices = noun_indices[:int(abs(round(len(noun_indices) * highest_score, 0)))]
    
    # calculate the distances of the intensity score and the highest emotion score
    distances = [round(abs(row["intensity"] -highest_score), 3) for index, row in lex.iterrows()]
    lex["distances"] = distances
    lex = lex.sort_values("distances", ascending=True)
    
    # get the new adjectives and make replacement dictionary from indices and text
    adjectives = lex["text"][:len(noun_indices)].tolist()
    adjectives = dict(zip(noun_indices, adjectives))
    # order the entries in descending order or else the replacement in output will shift
    adjectives = collections.OrderedDict(sorted(adjectives.items(), reverse=True))
    
    
    # make output to list, insert adjectives and convert back to string 
    bot_output = bot_output.split(" ")
    for index, adjective in adjectives.items(): 
        bot_output.insert(index, adjective)
    
    return " ".join(bot_output)
    
    
bot_output = "My car is a house with lights"
print(insert_adjectives(bot_output, lexicon, lexicon_text_list, lexicon_intensity_list, 0.32))

3
1.0
[1]
My visionary car is a soulful house with satisfied lights
